In [ ]:
%%writefile coalesced.cu
#include <cuda_runtime.h>
#include <iostream>

__global__ void coalescedKernel(const float* A, const float* B, float* C, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Bounds check
    if (idx < N) {
        C[idx] = A[idx] + B[idx];
    }
}

int main() {
    int N = 1 << 24;  // ~16 million elements
    size_t size = N * sizeof(float);

    float *hA = new float[N];
    float *hB = new float[N];
    float *hC = new float[N];

    for (int i = 0; i < N; i++) {
        hA[i] = 1.0f;
        hB[i] = 2.0f;
    }

    float *dA, *dB, *dC;
    cudaMalloc(&dA, size);
    cudaMalloc(&dB, size);
    cudaMalloc(&dC, size);

    cudaMemcpy(dA, hA, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dB, hB, size, cudaMemcpyHostToDevice);

    dim3 block(256);
    dim3 grid((N + block.x - 1) / block.x);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    coalescedKernel<<<grid, block>>>(dA, dB, dC, N);
    cudaEventRecord(stop);

    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    std::cout << "Coalesced kernel time: " << ms << " ms\n";

    cudaFree(dA);
    cudaFree(dB);
    cudaFree(dC);
    delete[] hA;
    delete[] hB;
    delete[] hC;

    return 0;
}


Writing coalesced.cu


In [ ]:
!nvcc coalesced.cu -o coalesced \
  -gencode arch=compute_75,code=sm_75 \
  -Xptxas -v

!./coalesced


ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z15coalescedKernelPKfS0_Pfi' for 'sm_75'
ptxas info    : Function properties for _Z15coalescedKernelPKfS0_Pfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 12 registers, 380 bytes cmem[0]
Coalesced kernel time: 0.880576 ms


In [ ]:
%%writefile non_coalesced.cu
#include <cuda_runtime.h>
#include <iostream>

#define STRIDE 8

__global__ void nonCoalescedKernel(const float* A, const float* B, float* C, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int accessIdx = idx * STRIDE;

    if (accessIdx < N) {
        C[accessIdx] = A[accessIdx] + B[accessIdx];
    }
}

int main() {
    int N = 1 << 24;
    size_t size = N * sizeof(float);

    float *hA = new float[N];
    float *hB = new float[N];
    float *hC = new float[N];

    for (int i = 0; i < N; i++) {
        hA[i] = 1.0f;
        hB[i] = 2.0f;
    }

    float *dA, *dB, *dC;
    cudaMalloc(&dA, size);
    cudaMalloc(&dB, size);
    cudaMalloc(&dC, size);

    cudaMemcpy(dA, hA, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dB, hB, size, cudaMemcpyHostToDevice);

    dim3 block(256);
    dim3 grid((N / STRIDE + block.x - 1) / block.x);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    nonCoalescedKernel<<<grid, block>>>(dA, dB, dC, N);
    cudaEventRecord(stop);

    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    std::cout << "Non-coalesced kernel time: " << ms << " ms\n";

    cudaFree(dA);
    cudaFree(dB);
    cudaFree(dC);
    delete[] hA;
    delete[] hB;
    delete[] hC;

    return 0;
}


Writing non_coalesced.cu


In [ ]:
!nvcc non_coalesced.cu -o non_coalesced \
  -gencode arch=compute_75,code=sm_75 \
  -Xptxas -v

!./non_coalesced

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z18nonCoalescedKernelPKfS0_Pfi' for 'sm_75'
ptxas info    : Function properties for _Z18nonCoalescedKernelPKfS0_Pfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 12 registers, 380 bytes cmem[0]
Non-coalesced kernel time: 1.10883 ms


In [ ]:
%%writefile shared_memory.cu
#include <cuda_runtime.h>
#include <iostream>

#define BLOCK_SIZE 256

// Baseline: Global Memory Reduction
__global__ void globalReduce(const float* input, float* output, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < N) {
        atomicAdd(output, input[idx]);
    }
}

// Optimized: Shared Memory Reduction

__global__ void sharedReduce(const float* input, float* output, int N) {
    __shared__ float sdata[BLOCK_SIZE];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + tid;

    // Load from global memory to shared memory
    if (idx < N)
        sdata[tid] = input[idx];
    else
        sdata[tid] = 0.0f;

    __syncthreads();

    // Reduction in shared memory
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            sdata[tid] += sdata[tid + stride];
        }
        __syncthreads();
    }

    // One atomic add per block
    if (tid == 0) {
        atomicAdd(output, sdata[0]);
    }
}

// Host Code
int main() {
    int N = 1 << 20;  // ~1 million elements
    size_t size = N * sizeof(float);

    float* h_input = new float[N];
    for (int i = 0; i < N; i++)
        h_input[i] = 1.0f;

    float *d_input, *d_output;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, sizeof(float));

    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

    dim3 block(BLOCK_SIZE);
    dim3 grid((N + BLOCK_SIZE - 1) / BLOCK_SIZE);

    // ---------------- Global Memory Version ----------------
    cudaMemset(d_output, 0, sizeof(float));

    cudaEvent_t start1, stop1;
    cudaEventCreate(&start1);
    cudaEventCreate(&stop1);

    cudaEventRecord(start1);
    globalReduce<<<grid, block>>>(d_input, d_output, N);
    cudaEventRecord(stop1);

    cudaEventSynchronize(stop1);

    float time_global;
    cudaEventElapsedTime(&time_global, start1, stop1);

    // ---------------- Shared Memory Version ----------------
    cudaMemset(d_output, 0, sizeof(float));

    cudaEvent_t start2, stop2;
    cudaEventCreate(&start2);
    cudaEventCreate(&stop2);

    cudaEventRecord(start2);
    sharedReduce<<<grid, block>>>(d_input, d_output, N);
    cudaEventRecord(stop2);

    cudaEventSynchronize(stop2);

    float time_shared;
    cudaEventElapsedTime(&time_shared, start2, stop2);

    // ---------------- Results ----------------
    std::cout << "Global memory kernel time:  " << time_global << " ms\n";
    std::cout << "Shared memory kernel time:  " << time_shared << " ms\n";
    std::cout << "Speedup: " << time_global / time_shared << "x\n";

    cudaFree(d_input);
    cudaFree(d_output);
    delete[] h_input;

    return 0;
}


Writing shared_memory.cu


In [ ]:
!nvcc shared_memory.cu -o shared_memory \
  -gencode arch=compute_75,code=sm_75 \
  -Xptxas -v

!./shared_memory

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z12sharedReducePKfPfi' for 'sm_75'
ptxas info    : Function properties for _Z12sharedReducePKfPfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 10 registers, 1024 bytes smem, 372 bytes cmem[0]
ptxas info    : Compiling entry function '_Z12globalReducePKfPfi' for 'sm_75'
ptxas info    : Function properties for _Z12globalReducePKfPfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 6 registers, 372 bytes cmem[0]
Global memory kernel time:  3.77238 ms
Shared memory kernel time:  0.122784 ms
Speedup: 30.7237x


In [ ]:
# #cpu_baseline
# import torch
# import torch.nn as nn
# import time

# # -------------------------------------------------
# # Force CPU
# # -------------------------------------------------
# device = torch.device("cpu")

# # -------------------------------------------------
# # Tiny U-Net Model
# # -------------------------------------------------
# class TinyUNet(nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.enc1 = nn.Sequential(
#             nn.Conv2d(1, 8, 3, padding=1),
#             nn.ReLU(),
#             nn.Conv2d(8, 8, 3, padding=1),
#             nn.ReLU()
#         )

#         self.pool = nn.MaxPool2d(2)

#         self.bottleneck = nn.Sequential(
#             nn.Conv2d(8, 16, 3, padding=1),
#             nn.ReLU(),
#             nn.Conv2d(16, 16, 3, padding=1),
#             nn.ReLU()
#         )

#         self.up = nn.Upsample(scale_factor=2, mode='nearest')

#         self.dec1 = nn.Sequential(
#             nn.Conv2d(16, 8, 3, padding=1),
#             nn.ReLU(),
#             nn.Conv2d(8, 8, 3, padding=1),
#             nn.ReLU()
#         )

#         self.out = nn.Conv2d(8, 1, 1)

#     def forward(self, x):
#         x1 = self.enc1(x)
#         x2 = self.pool(x1)
#         x3 = self.bottleneck(x2)
#         x4 = self.up(x3)
#         x5 = self.dec1(x4)
#         return self.out(x5)

# # -------------------------------------------------
# # Input: Grayscale Medical Image
# # -------------------------------------------------
# input_image = torch.randn(1, 1, 256, 256, device=device)

# model = TinyUNet().to(device)
# model.eval()

# # -------------------------------------------------
# # CPU Timing
# # -------------------------------------------------
# with torch.no_grad():
#     start = time.time()
#     output = model(input_image)
#     end = time.time()

# print(f"CPU inference time: {end - start:.4f} seconds")
# print(f"Output shape: {output.shape}")


In [ ]:
# #gpu_acc
# import torch
# import torch.nn as nn
# import time

# # -------------------------------------------------
# # GPU Device
# # -------------------------------------------------
# device = torch.device("cuda")

# # -------------------------------------------------
# # Tiny U-Net Model (UNCHANGED)
# # -------------------------------------------------
# class TinyUNet(nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.enc1 = nn.Sequential(
#             nn.Conv2d(1, 8, 3, padding=1),
#             nn.ReLU(),
#             nn.Conv2d(8, 8, 3, padding=1),
#             nn.ReLU()
#         )

#         self.pool = nn.MaxPool2d(2)

#         self.bottleneck = nn.Sequential(
#             nn.Conv2d(8, 16, 3, padding=1),
#             nn.ReLU(),
#             nn.Conv2d(16, 16, 3, padding=1),
#             nn.ReLU()
#         )

#         self.up = nn.Upsample(scale_factor=2, mode='nearest')

#         self.dec1 = nn.Sequential(
#             nn.Conv2d(16, 8, 3, padding=1),
#             nn.ReLU(),
#             nn.Conv2d(8, 8, 3, padding=1),
#             nn.ReLU()
#         )

#         self.out = nn.Conv2d(8, 1, 1)

#     def forward(self, x):
#         x1 = self.enc1(x)
#         x2 = self.pool(x1)
#         x3 = self.bottleneck(x2)
#         x4 = self.up(x3)
#         x5 = self.dec1(x4)
#         return self.out(x5)

# # -------------------------------------------------
# # Input Image
# # -------------------------------------------------
# input_image = torch.randn(1, 1, 256, 256, device=device)

# model = TinyUNet().to(device)
# model.eval()

# # Warm-up (important for fair GPU timing)
# with torch.no_grad():
#     for _ in range(5):
#         _ = model(input_image)

# torch.cuda.synchronize()

# # -------------------------------------------------
# # GPU Timing (CUDA events)
# # -------------------------------------------------
# start = torch.cuda.Event(enable_timing=True)
# end = torch.cuda.Event(enable_timing=True)

# with torch.no_grad():
#     start.record()
#     output = model(input_image)
#     end.record()

# torch.cuda.synchronize()

# print(f"GPU inference time: {start.elapsed_time(end):.4f} ms")
# print(f"Output shape: {output.shape}")


In [ ]:
import numpy as np
import time

# Dimensions
H, W = 256, 256
C1, C2 = 4, 8
K = 3

# Input image
input_img = np.ones((1, H, W), dtype=np.float32)

# Weights
W1 = np.random.randn(C1, 1, K, K).astype(np.float32)
W2 = np.random.randn(C2, C1, K, K).astype(np.float32)
W3 = np.random.randn(C1, C2, K, K).astype(np.float32)

# Helper functions
def conv_relu(inp, W):
    C_out, C_in, _, _ = W.shape
    _, H, Wd = inp.shape
    out = np.zeros((C_out, H, Wd), dtype=np.float32)

    for oc in range(C_out):
        for y in range(H):
            for x in range(Wd):
                acc = 0.0
                for ic in range(C_in):
                    for ky in range(-1, 2):
                        for kx in range(-1, 2):
                            iy, ix = y + ky, x + kx
                            if 0 <= iy < H and 0 <= ix < Wd:
                                acc += inp[ic, iy, ix] * W[oc, ic, ky+1, kx+1]
                out[oc, y, x] = max(acc, 0.0)
    return out

def maxpool(inp):
    C, H, W = inp.shape
    out = np.zeros((C, H//2, W//2), dtype=np.float32)
    for c in range(C):
        for y in range(0, H, 2):
            for x in range(0, W, 2):
                out[c, y//2, x//2] = np.max(inp[c, y:y+2, x:x+2])
    return out

def upsample(inp):
    C, H, W = inp.shape
    out = np.zeros((C, H*2, W*2), dtype=np.float32)
    for c in range(C):
        for y in range(H):
            for x in range(W):
                out[c, y*2:y*2+2, x*2:x*2+2] = inp[c, y, x]
    return out


# CPU Timing
start = time.time()

enc1 = conv_relu(input_img, W1)
skip = enc1.copy()
enc2 = maxpool(enc1)
enc3 = conv_relu(enc2, W2)
dec1 = upsample(enc3)
dec2 = conv_relu(dec1, W3)
output = dec2 + skip

end = time.time()

print(f"[CPU] U-Net forward time: {end - start:.3f} s")
print("Output shape:", output.shape)


[CPU] U-Net forward time: 16.587 s
Output shape: (4, 256, 256)


In [ ]:
%%writefile gpu1.cu
#include <cuda_runtime.h>
#include <iostream>

#define H 256
#define W 256
#define C1 4
#define C2 8
#define BX 16
#define BY 16

// --------------------------------------------------
// Conv + ReLU kernel
// --------------------------------------------------
__global__ void conv_relu(
    const float* input,
    const float* weight,
    float* output,
    int Ht, int Wt, int Cin, int Cout
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int oc = blockIdx.z;

    if (x >= Wt || y >= Ht) return;

    float sum = 0.0f;

    for (int ic = 0; ic < Cin; ic++) {
        for (int ky = -1; ky <= 1; ky++) {
            for (int kx = -1; kx <= 1; kx++) {
                int ix = x + kx;
                int iy = y + ky;
                if (ix >= 0 && ix < Wt && iy >= 0 && iy < Ht) {
                    float val = input[ic * Ht * Wt + iy * Wt + ix];
                    float w = weight[
                        oc * Cin * 9 +
                        ic * 9 +
                        (ky + 1) * 3 + (kx + 1)
                    ];
                    sum += val * w;
                }
            }
        }
    }

    output[oc * Ht * Wt + y * Wt + x] = (sum > 0.0f) ? sum : 0.0f;
}

// --------------------------------------------------
// 2×2 MaxPool
// --------------------------------------------------
__global__ void maxpool(
    const float* input,
    float* output,
    int Cin
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int c = blockIdx.z;

    int ix = x * 2;
    int iy = y * 2;

    if (ix + 1 >= W || iy + 1 >= H) return;

    float m = input[c * H * W + iy * W + ix];
    m = fmaxf(m, input[c * H * W + iy * W + ix + 1]);
    m = fmaxf(m, input[c * H * W + (iy + 1) * W + ix]);
    m = fmaxf(m, input[c * H * W + (iy + 1) * W + ix + 1]);

    output[c * (H/2) * (W/2) + y * (W/2) + x] = m;
}

// --------------------------------------------------
// Upsampling (nearest neighbor)
// --------------------------------------------------
__global__ void upsample(
    const float* input,
    float* output,
    int Cin
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int c = blockIdx.z;

    if (x >= W || y >= H) return;

    output[c * H * W + y * W + x] =
        input[c * (H/2) * (W/2) + (y/2) * (W/2) + (x/2)];
}

// --------------------------------------------------
// MAIN
// --------------------------------------------------
int main() {

    size_t s1 = 1 * H * W * sizeof(float);
    size_t s4 = C1 * H * W * sizeof(float);
    size_t s8 = C2 * (H/2) * (W/2) * sizeof(float);

    float *d_in, *d_e1, *d_skip, *d_p, *d_e2, *d_up, *d_out;
    float *w1, *w2, *w3;

    cudaMalloc(&d_in, s1);
    cudaMalloc(&d_e1, s4);
    cudaMalloc(&d_skip, s4);
    cudaMalloc(&d_p, C1 * (H/2) * (W/2) * sizeof(float));
    cudaMalloc(&d_e2, s8);
    cudaMalloc(&d_up, s4);
    cudaMalloc(&d_out, s4);

    cudaMalloc(&w1, C1 * 9 * sizeof(float));
    cudaMalloc(&w2, C2 * C1 * 9 * sizeof(float));
    cudaMalloc(&w3, C1 * C2 * 9 * sizeof(float));

    cudaMemset(d_in, 1, s1);

    dim3 block(BX, BY);
    dim3 grid1((W + BX - 1) / BX, (H + BY - 1) / BY, C1);
    dim3 grid2((W/2 + BX - 1) / BX, (H/2 + BY - 1) / BY, C2);

    // ---------------- Warm-up ----------------
    conv_relu<<<grid1, block>>>(d_in, w1, d_e1, H, W, 1, C1);
    cudaDeviceSynchronize();

    // ---------------- Timing ----------------
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);

    // Encoder
    conv_relu<<<grid1, block>>>(d_in, w1, d_e1, H, W, 1, C1);
    cudaMemcpy(d_skip, d_e1, s4, cudaMemcpyDeviceToDevice);

    maxpool<<<dim3((W/2+BX-1)/BX, (H/2+BY-1)/BY, C1), block>>>(d_e1, d_p, C1);

    conv_relu<<<grid2, block>>>(d_p, w2, d_e2, H/2, W/2, C1, C2);

    // Decoder
    upsample<<<grid1, block>>>(d_e2, d_up, C2);
    conv_relu<<<grid1, block>>>(d_up, w3, d_out, H, W, C2, C1);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float time_ms;
    cudaEventElapsedTime(&time_ms, start, stop);

    std::cout << "[GPU] U-Net forward pass time: "
              << time_ms << " ms\n";

    cudaFree(d_in); cudaFree(d_e1); cudaFree(d_skip);
    cudaFree(d_p);  cudaFree(d_e2); cudaFree(d_up); cudaFree(d_out);
    cudaFree(w1); cudaFree(w2); cudaFree(w3);

    return 0;
}

Writing gpu1.cu


In [ ]:
!nvcc gpu1.cu -o gpu1 \
  -gencode arch=compute_75,code=sm_75 \
  -Xptxas -v

!./gpu1

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z8upsamplePKfPfi' for 'sm_75'
ptxas info    : Function properties for _Z8upsamplePKfPfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 9 registers, 372 bytes cmem[0]
ptxas info    : Compiling entry function '_Z7maxpoolPKfPfi' for 'sm_75'
ptxas info    : Function properties for _Z7maxpoolPKfPfi
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 14 registers, 372 bytes cmem[0]
ptxas info    : Compiling entry function '_Z9conv_reluPKfS0_Pfiiii' for 'sm_75'
ptxas info    : Function properties for _Z9conv_reluPKfS0_Pfiiii
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 40 registers, 392 bytes cmem[0]
[GPU] U-Net forward pass time: 0.948448 ms
